In [1]:
pip install wn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import wn
import torch
from transformers import pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import string
wn.add("own-pt.tar.gz")

/home/paula/anaconda3/envs/ambiguidade_env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/paula/anaconda3/envs/ambiguidade_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping own-pt:1.0.0 (OpenWordnet-PT); already added



In [3]:
def pre_processamento(frase):
  frase = frase.lower()
  frase = frase.translate(str.maketrans('', '', string.punctuation))
  return frase

In [4]:
class WSD:
    def __init__(self):
        self.extractor = pipeline(
            "feature-extraction",
            model="neuralmind/bert-base-portuguese-cased",
            tokenizer="neuralmind/bert-base-portuguese-cased",
            framework="pt"
        )

    def gerarEmbeddingSentido(self, sentido):
        #Combinando token CLS e a media dos embeddings do sentido
        embs = self.extractor(sentido)[0]
        embs = np.array(embs)

        embscls = embs[0]
        embssentido = np.mean(embs[1:-1], axis=0)

        return  (0.8 * embscls + 0.2 * embssentido)
        #Usando so token CLS
        #  embs = self.extractor(sentido)[0]
        #  return embs[0]

    def gerarEmbeddingsPalavra(self, contexto, palavra):
        tkscontexto = self.extractor.tokenizer.tokenize(contexto)
        tkspalavra = self.extractor.tokenizer.tokenize(palavra)

        indicespalavra = self.acharPalavraContexto(tkscontexto, tkspalavra)

        if not indicespalavra:
            print(f"Palavra '{palavra}' não encontrada no contexto")
            return None

        embscontexto = self.extractor(contexto)[0]

        indicespalavrasajustado = [i + 1 for i in indicespalavra]

        embspalavra = [embscontexto[i] for i in indicespalavrasajustado]

        return np.mean(embspalavra, axis=0)

    def acharPalavraContexto(self, tkscontexto, tkspalavra):
        for i in range(len(tkscontexto) - len(tkspalavra) + 1):
            if tkscontexto[i:i+len(tkspalavra)] == tkspalavra:
                return list(range(i, i + len(tkspalavra)))
        return []

    def compararPalavraSentido(self, contexto, palavra, sentido):
        embspalavra = self.gerarEmbeddingsPalavra(contexto, palavra)
        embssentido = self.gerarEmbeddingSentido(sentido)
        if embspalavra is None:
            return 0.0
        return cosine_similarity([embspalavra], [embssentido])[0][0]

In [5]:
class OWNPT:
    def getSenses(self, word):
        senses = wn.senses(word)
        sensesstr = []
        for sense in senses:
            definicao = sense.synset().definition()
            if definicao:
                sensesstr.append(definicao)
        return sensesstr

In [6]:
wsd = WSD()
ownpt = OWNPT()

nomes_das_colunas = ['frase', 'palavra', 'sentido']
corpus = pd.read_csv('corpus2teste_revisado_multi.csv', sep=',', on_bad_lines='skip', names=nomes_das_colunas)
corpus['sentido'] = corpus['sentido'].apply(lambda x: x.split('|'))
pd.set_option('display.max_colwidth', None)
corpus.head()
y_true = []
y_pred = []

for idx, row in corpus.iterrows():
    frase = pre_processamento(row['frase'])
    palavra = row['palavra']
    sentidosreais = row['sentido']

    maior_sim = -1
    melhor_sentido = None

    senses = ownpt.getSenses(palavra)

    for sentido_candidato in senses:
        sim = wsd.compararPalavraSentido(frase, palavra, pre_processamento(sentido_candidato))
        if sim is not None and sim > maior_sim:
            maior_sim = sim
            melhor_sentido = sentido_candidato

    if melhor_sentido:
        if melhor_sentido in sentidosreais:
            y_true.append(melhor_sentido)
        else:
            y_true.append(sentidosreais[0])
        y_pred.append(melhor_sentido)

#print("Relatório de classificação:")
#print(classification_report(y_true, y_pred, zero_division=0))

accuracy = accuracy_score(y_true, y_pred)
print(f"Acurácia: {accuracy:.2f}")

precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
print(f"Precisão (macro): {precision:.2f}")
print(f"Revocação (macro): {recall:.2f}")
print(f"F1-score (macro): {f1:.2f}")

Device set to use cpu


Acurácia: 0.67
Precisão (macro): 0.60
Revocação (macro): 0.66
F1-score (macro): 0.59


In [9]:
# Pré Processamento Lesk (remove stopwords)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('portuguese'))

def pre_processamento(frase):
    frase = frase.lower()
    frase = frase.translate(str.maketrans('', '', string.punctuation))
    tokens = frase.split()
    return [w for w in tokens if w not in stop_words]

[nltk_data] Downloading package stopwords to /home/paula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Lesk
def lesk(palavra, frase, lexico):
    senses = lexico.getSenses(palavra)
    contexto = set(pre_processamento(frase))

    best_sense = None
    max_overlap = -1

    for sense in senses:
        gloss_words = set(sense.lower().split())
        overlap = len(contexto.intersection(gloss_words))

        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense

    return best_sense

corpus = pd.read_csv('corpus2teste_revisado_multi.csv', sep=',', on_bad_lines='skip')
corpus.columns = ['frase', 'palavra', 'sentido']
corpus['sentido'] = corpus['sentido'].apply(lambda x: x.split('|'))

y_true = []
y_pred = []
resultados = []

for _, row in corpus.iterrows():
    frase = row['frase']
    palavra = row['palavra']
    sentidosreais = row['sentido']

    sentido_predito = lesk(palavra, frase, ownpt)
    if sentido_predito:
        if sentido_predito in sentidosreais:
            y_true.append(sentido_predito)
        else:
            y_true.append(sentidosreais[0])
        y_pred.append(sentido_predito)

    resultados.append({
            'frase': frase,
            'palavra': palavra,
            'sentidos_reais': ' | '.join(sentidosreais),
            'sentido_previsto': melhor_sentido,
            'similaridade': round(maior_sim, 4)
        })

df_resultados = pd.DataFrame(resultados)
#print("Relatório de classificação:")
#print(classification_report(y_true, y_pred, zero_division=0))

# Cálculo das métricas
print("--Lesk")
print("Acurácia:", accuracy_score(y_true, y_pred))

precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
print(f"Precisão (macro): {precision:.2f}")
print(f"Revocação (macro): {recall:.2f}")
print(f"F1-score (macro): {f1:.2f}")

--Lesk
Acurácia: 0.41700404858299595
Precisão (macro): 0.26
Revocação (macro): 0.41
F1-score (macro): 0.28


In [12]:
# Filtrar resultados onde o sentido real é diferente do previsto
df_diferentes = df_resultados[df_resultados.apply(lambda row: row['sentido_previsto'] not in row['sentidos_reais'], axis=1)].copy()

# Ordenar os resultados filtrados pela similaridade em ordem decrescente
df_diferentes_ordenado = df_diferentes.sort_values(by='similaridade', ascending=False)

# Exibir o DataFrame filtrado e ordenado
print("\nResultados onde o sentido real é diferente do previsto, ordenados pela maior similaridade:")
df_diferentes_ordenado


Resultados onde o sentido real é diferente do previsto, ordenados pela maior similaridade:


,frase,palavra,sentidos_reais,sentido_previsto,similaridade
493,As ondas de rádio podem viajar por longas distâncias antes de serem captadas.,viajar,viajar atraves ou sobre uma area,ir a certos lugares para passear,0.2073
0,Ele esperou por mais de uma hora no banco desconfortável do lado de fora do consultório.,banco,um assento longo para mais de uma pessoa,ir a certos lugares para passear,0.2073
1,O banco de madeira da varanda precisa de uma nova camada de verniz.,banco,um assento longo para mais de uma pessoa,ir a certos lugares para passear,0.2073
2,Preciso ir ao banco para sacar dinheiro antes da viagem.,banco,uma instituição financeira que aceita depósitos e canaliza o dinheiro em atividades de empréstimo,ir a certos lugares para passear,0.2073
3,O banco central anunciou que a taxa de juros permanecerá estável este mês.,banco,uma instituição financeira que aceita depósitos e canaliza o dinheiro em atividades de empréstimo,ir a certos lugares para passear,0.2073
...,...,...,...,...,...
14,Havia uma pequena inscrição quase ilegível no fundo da antiga caixa de madeira.,fundo,o lado inferior de qualquer coisa,ir a certos lugares para passear,0.2073
13,Ele procurou as chaves do carro no fundo da mochila e não as encontrou.,fundo,o lado inferior de qualquer coisa,ir a certos lugares para passear,0.2073
12,Toda a sua teoria foi construída com base em premissas que se provaram falsas.,base,os pressupostos fundamentais dos quais algo é iniciado ou desenvolvido ou calculado ou explicado,ir a certos lugares para passear,0.2073
11,O cálculo da estimativa populacional teve como base os dados do último censo.,base,os pressupostos fundamentais dos quais algo é iniciado ou desenvolvido ou calculado ou explicado,ir a certos lugares para passear,0.2073


In [16]:
!python -m spacy download pt_core_news_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/paula/anaconda3/envs/ambiguidade_env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 4.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [19]:
import spacy
nlp = spacy.load("pt_core_news_sm")


def selecionarSentido(contexto, palavra, wsd, ownpt):
    doc = nlp(palavra)
    palavra = doc[0].lemma
    sentidos = ownpt.getSenses(palavra)

    print('Palavra', palavra)
    print('Sentidos', sentidos)

    # Para ser ambígua, uma palavra precisa ter dois ou mais sentidos
    if (sentidos == None or (sentidos != None and len(sentidos) <= 1)):
        return None

    maior_sim = -1
    melhor_sentido = None

    for sentido in sentidos:
        sim = wsd.compararPalavraSentido(contexto, palavra, pre_processamento(sentido))
        if sim is not None and sim > maior_sim:
            maior_sim = sim
            melhor_sentido = sentido

    return melhor_sentido
